In [1]:
import os
import sys
print(f"{sys.executable = }")
print(f"{os.getcwd() = }")
sys.path.append('/nas/home/sh0416/sentence-benchmark/src')


sys.executable = '/home/sh0416/anaconda3/envs/sentence-benchmark/bin/python'
os.getcwd() = '/nas/home/sh0416/sentence-benchmark/src/notebooks'


In [2]:
root = "/nas/home/sh0416/data/semeval16/task2/test_goldStandard"
model_args_path = "/home/sh0416/checkpoints/20211021_113441/model_args.json"
checkpoint_path = "/home/sh0416/checkpoints/20211021_113441/checkpoint-1500/pytorch_model.bin"
sources = ["images", "headlines", "answers-students"]

filepaths_sent1 = [f"STSint.testinput.{source}.sent1.txt" for source in sources]
filepaths_sent2 = [f"STSint.testinput.{source}.sent2.txt" for source in sources]
filepaths_sent1_tok = [f"STSint.testinput.{source}.sent1.chunk.txt" for source in sources]
filepaths_sent2_tok = [f"STSint.testinput.{source}.sent2.chunk.txt" for source in sources]
filepaths_align = [f"STSint.testinput.{source}.wa" for source in sources]

In [3]:
import os
from simcse.eval.ists import load_alignment
from simcse.eval.ists import save_alignment
data = load_alignment(os.path.join(root, filepaths_align[0]))
save_alignment(data, "test.wa")

In [6]:
from simcse.eval.ists import load_instances
instances = load_instances(
    os.path.join(root, filepaths_sent1[2]),
    os.path.join(root, filepaths_sent2[2]),
    os.path.join(root, filepaths_sent1_tok[2]),
    os.path.join(root, filepaths_sent2_tok[2])
)
instances[0]

{'id': 1,
 'sent1': 'both bulbs a and c still have a closed path',
 'sent2': 'Bulbs A and C are still in closed paths',
 'sent1_chunk': ['both', 'bulbs a and c', 'still', 'have', 'a closed path'],
 'sent2_chunk': ['Bulbs A and C', 'are', 'still', 'in closed paths']}

In [7]:
import json
from transformers import AutoTokenizer
from simcse.config import ModelArguments

with open(model_args_path) as f:
    model_args = ModelArguments(**json.load(f))
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, use_fast=False)

In [8]:
model_args

ModelArguments(model_name_or_path='bert-base-uncased', temp=0.05, pooler_type='cls', hidden_dropout_prob=0.1, mlp_only_train=True, coeff_mlm=0.1, loss_rwmd=True, layer_idx=12)

In [9]:
from simcse.eval.ists import preprocess_instances

prep_instances = preprocess_instances(tokenizer, instances)
prep_instances[0]

{'instance': {'id': 1,
  'sent1': 'both bulbs a and c still have a closed path',
  'sent2': 'Bulbs A and C are still in closed paths',
  'sent1_chunk': ['both', 'bulbs a and c', 'still', 'have', 'a closed path'],
  'sent2_chunk': ['Bulbs A and C', 'are', 'still', 'in closed paths']},
 'sent1_token': ['both',
  'bulbs',
  'a',
  'and',
  'c',
  'still',
  'have',
  'a',
  'closed',
  'path'],
 'sent2_token': ['bulbs',
  'a',
  'and',
  'c',
  'are',
  'still',
  'in',
  'closed',
  'paths'],
 'inputs1': {'input_ids': tensor([[  101,  2119, 25548,  1037,  1998,  1039,  2145,  2031,  1037,  2701,
           4130,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 'inputs2': {'input_ids': tensor([[  101, 25548,  1037,  1998,  1039,  2024,  2145,  1999,  2701, 10425,
            102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
import torch
from simcse.models import create_contrastive_learning

module = create_contrastive_learning(model_args)
module.load_state_dict(torch.load(checkpoint_path))
module.model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentenceSimilarityModel(
  (representation_model): LastHiddenSentenceRepresentationModel(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_featur

In [11]:
from simcse.eval.ists import inference
import logging

logging.basicConfig(level=logging.DEBUG)
infered_instances = inference(module.model, prep_instances)

DEBUG:simcse.eval.ists:{'instance': {'id': 1, 'sent1': 'both bulbs a and c still have a closed path', 'sent2': 'Bulbs A and C are still in closed paths', 'sent1_chunk': ['both', 'bulbs a and c', 'still', 'have', 'a closed path'], 'sent2_chunk': ['Bulbs A and C', 'are', 'still', 'in closed paths']}, 'sent1_token': ['both', 'bulbs', 'a', 'and', 'c', 'still', 'have', 'a', 'closed', 'path'], 'sent2_token': ['bulbs', 'a', 'and', 'c', 'are', 'still', 'in', 'closed', 'paths'], 'inputs1': {'input_ids': tensor([[  101,  2119, 25548,  1037,  1998,  1039,  2145,  2031,  1037,  2701,
          4130,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}, 'inputs2': {'input_ids': tensor([[  101, 25548,  1037,  1998,  1039,  2024,  2145,  1999,  2701, 10425,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}}
DEBUG:simcse

In [12]:
infered_instances[0]["pairs"]

[{'sent1_word_ids': [2, 3, 4, 5],
  'sent2_word_ids': [1, 2, 3, 4],
  'type': 'EQUI',
  'score': 5,
  'comment': ''},
 {'sent1_word_ids': [6],
  'sent2_word_ids': [6],
  'type': 'EQUI',
  'score': 5,
  'comment': ''},
 {'sent1_word_ids': [7],
  'sent2_word_ids': [5],
  'type': 'EQUI',
  'score': 5,
  'comment': ''},
 {'sent1_word_ids': [8, 9, 10],
  'sent2_word_ids': [7, 8, 9],
  'type': 'EQUI',
  'score': 5,
  'comment': ''}]

In [13]:
from simcse.eval.ists import save_infered_instances

save_infered_instances(infered_instances, f"{sources[2]}_{model_args.model_name_or_path}.wa")

In [ ]:
x = np.zeros((4, 2))
x[[0, 1], [0, 1]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_heatmap(sentence):
    source = ['0 NIL : '] + sentence["source"]
    translation = ['0 NIL : '] + sentence["translation"]
    heatmap = np.zeros((len(source), len(translation)))
    for a in sentence["alignment"]:
        if a["score"] != 'NIL':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            score = float(a["score"])
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    heatmap[token_id1, token_id2] = score

    fig, ax = plt.subplots(facecolor='white')
    ax.set_ylabel("source")
    ax.set_yticks(np.arange(0.5, len(source)+0.5, 1))
    ax.set_yticklabels(source)
    ax.set_xlabel("translation")
    ax.set_xticks(np.arange(0.5, len(translation)+0.5, 1))
    ax.set_xticklabels(translation, rotation=90)
    ax.pcolormesh(heatmap, cmap=plt.get_cmap('Blues'))
    ax.invert_yaxis()
    for a in sentence["alignment"]:
        if a["type"] != 'NOALI':
            token_id_seq1, token_id_seq2 = a["pair_id"]
            for token_id1 in token_id_seq1:
                for token_id2 in token_id_seq2:
                    ax.text(token_id2, token_id1+0.5, a["type"], c='w')
    plt.tight_layout()
    return fig, ax

for sentence in data[:100]:
    plot_heatmap(sentence)
    plt.show()

In [ ]:
from simcse.models import create_contrastive_learning
from simcse.config import ModelArguments

model_args = ModelArguments(loss_rwmd=True)
module = create_contrastive_learning(model_args)